<a href="https://colab.research.google.com/github/pratx08/Gen-AI/blob/main/Gen_AI__Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install evaluate -q

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)
import evaluate
from transformers import logging

# Silence warnings for cleaner output
logging.set_verbosity_error()

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [44]:
# Load the SQuAD v1.1 dataset
dataset = load_dataset("squad")

# Take a small subset for quick experimentation
train_size, val_size = 2000, 500
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(train_size))
small_val_dataset = dataset["validation"].shuffle(seed=42).select(range(val_size))

print("Dataset splits:", dataset.keys())
print("Example training record:", small_train_dataset[0])


Dataset splits: dict_keys(['train', 'validation'])
Example training record: {'id': '573173d8497a881900248f0c', 'title': 'Egypt', 'context': 'The Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery.', 'question': 'What percentage of Egyptians polled support death penalty for those leaving Islam?', 'answers': {'text': ['84%'], 'answer_start': [468]}}


In [45]:
# Load GPT-2 tokenizer
gpt2_model_name = "gpt2"
gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_model_name)

# Add a padding token (GPT-2 does not have one)
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# Preprocessing: concatenate question + context, target = answer
def preprocess_gpt2(examples):
    inputs = ["question: " + q + " context: " + c for q, c in zip(examples["question"], examples["context"])]
    targets = [a["text"][0] for a in examples["answers"]]  # take the first answer

    model_inputs = gpt2_tokenizer(
        inputs, text_target=targets,
        max_length=256, truncation=True, padding="max_length"
    )

    # Manually pad labels with -100 for ignored positions
    if "labels" in model_inputs:
        labels = model_inputs["labels"]
        for i in range(len(labels)):
            labels[i] = labels[i] + [-100] * (256 - len(labels[i])) if len(labels[i]) < 256 else labels[i][:256]
        model_inputs["labels"] = labels

    return model_inputs

# Apply preprocessing
train_gpt2 = small_train_dataset.map(preprocess_gpt2, batched=True, remove_columns=dataset["train"].column_names)
val_gpt2 = small_val_dataset.map(preprocess_gpt2, batched=True, remove_columns=dataset["validation"].column_names)

print("Sample tokenized GPT-2 input:")
print(gpt2_tokenizer.decode(train_gpt2[0]["input_ids"][:100]))


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Sample tokenized GPT-2 input:
question: What percentage of Egyptians polled support death penalty for those leaving Islam? context: The Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84


In [46]:
print("Input IDs and Labels dimensions for first 10 samples:")
for i in range(10):
    print(f"Sample {i}: Input IDs length = {len(train_gpt2[i]['input_ids'])}, Labels length = {len(train_gpt2[i]['labels'])}")


Input IDs and Labels dimensions for first 10 samples:
Sample 0: Input IDs length = 256, Labels length = 256
Sample 1: Input IDs length = 256, Labels length = 256
Sample 2: Input IDs length = 256, Labels length = 256
Sample 3: Input IDs length = 256, Labels length = 256
Sample 4: Input IDs length = 256, Labels length = 256
Sample 5: Input IDs length = 256, Labels length = 256
Sample 6: Input IDs length = 256, Labels length = 256
Sample 7: Input IDs length = 256, Labels length = 256
Sample 8: Input IDs length = 256, Labels length = 256
Sample 9: Input IDs length = 256, Labels length = 256


In [47]:
# Data collator
data_collator_gpt2 = DataCollatorForLanguageModeling(tokenizer=gpt2_tokenizer, mlm=False)

# Load GPT-2 model
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_model_name)

# Training arguments (compatible with your version)
training_args_gpt2 = TrainingArguments(
    output_dir="./gpt2-squad",
    eval_strategy="steps",              # <- fixed here
    eval_steps=100,
    logging_steps=100,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,                 # good default for GPT-2 finetuning
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=1,
    warmup_steps=50,
    gradient_accumulation_steps=4,
    fp16=torch.cuda.is_available(),
    report_to=[],                       # disable wandb / tensorboard
)

# Trainer
trainer_gpt2 = Trainer(
    model=gpt2_model,
    args=training_args_gpt2,
    train_dataset=train_gpt2,
    eval_dataset=val_gpt2,
    data_collator=data_collator_gpt2,
)

# Train GPT-2
# trainer_gpt2.train()



In [48]:
# Test the trained (or pre-trained) GPT-2 with a QA example
sample_input = "question: Who wrote the novel '1984'? context: The novel '1984' was written by George Orwell in 1949."
inputs = gpt2_tokenizer(sample_input, return_tensors="pt").to(device)

outputs = gpt2_model.generate(
    inputs["input_ids"],
    max_length=50,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95
)

print("Generated Answer:", gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True))


Generated Answer: question: Who wrote the novel '1984'? context: The novel '1984' was written by George Orwell in 1949. He was only 21-23, but he had never completed his thesis before he died. The author's name was George Orwell.


In [49]:
!pip install evaluate -q

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    DefaultDataCollator,
    Trainer,
    TrainingArguments,
)
import evaluate
from transformers import logging

logging.set_verbosity_error()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [50]:
# Load SQuAD v1.1
dataset = load_dataset("squad")

# Take smaller subset for faster experimentation
train_size, val_size = 2000, 500
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(train_size))
small_val_dataset = dataset["validation"].shuffle(seed=42).select(range(val_size))

print("Example training record:", small_train_dataset[0])


Example training record: {'id': '573173d8497a881900248f0c', 'title': 'Egypt', 'context': 'The Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery.', 'question': 'What percentage of Egyptians polled support death penalty for those leaving Islam?', 'answers': {'text': ['84%'], 'answer_start': [468]}}


In [51]:
bert_model_name = "bert-base-uncased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

max_length = 384
doc_stride = 128

def preprocess_bert(examples):
    inputs = bert_tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_mapping = inputs.pop("overflow_to_sample_mapping")

    start_positions, end_positions = [], []

    for i, offsets in enumerate(offset_mapping):
        input_ids = inputs["input_ids"][i]
        cls_index = input_ids.index(bert_tokenizer.cls_token_id)

        sequence_ids = inputs.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        answer_start = answers["answer_start"][0]
        answer_text = answers["text"][0]
        answer_end = answer_start + len(answer_text)

        # Find start token
        start_pos, end_pos = 0, 0
        if len(offsets) == 0:
            start_pos = cls_index
            end_pos = cls_index
        else:
            for idx, (start, end) in enumerate(offsets):
                if start <= answer_start < end:
                    start_pos = idx
                if start < answer_end <= end:
                    end_pos = idx
        start_positions.append(start_pos)
        end_positions.append(end_pos)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

train_bert = small_train_dataset.map(preprocess_bert, batched=True, remove_columns=dataset["train"].column_names)
val_bert = small_val_dataset.map(preprocess_bert, batched=True, remove_columns=dataset["validation"].column_names)

print("Keys after preprocessing:", train_bert.column_names)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Keys after preprocessing: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions']


In [52]:
# Data collator
data_collator_bert = DefaultDataCollator()

# Load BERT for Question Answering
bert_model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)

# Updated training arguments
training_args_bert = TrainingArguments(
    output_dir="./bert-squad",
    eval_strategy="steps",     # <- correct param name
    eval_steps=200,                  # evaluate every 200 steps
    logging_steps=100,               # log every 100 steps
    per_device_train_batch_size=8,   # larger batch for faster convergence
    per_device_eval_batch_size=8,
    num_train_epochs=2,              # train for 2 epochs
    learning_rate=3e-5,              # explicitly set LR
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=1,
    warmup_steps=100,                # slightly longer warmup
    fp16=torch.cuda.is_available(),  # mixed precision if GPU supports
    report_to=[],
)

# Trainer
trainer_bert = Trainer(
    model=bert_model,
    args=training_args_bert,
    train_dataset=train_bert,
    eval_dataset=val_bert,
    tokenizer=bert_tokenizer,
    data_collator=data_collator_bert,
)

# Train BERT
trainer_bert.train()



/tmp/ipython-input-2314550968.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_bert = Trainer(


{'loss': 4.9507, 'grad_norm': 13.477773666381836, 'learning_rate': 2.97e-05, 'epoch': 0.3968253968253968}
{'loss': 2.8581, 'grad_norm': 31.458402633666992, 'learning_rate': 2.264851485148515e-05, 'epoch': 0.7936507936507936}
{'eval_loss': 2.4321563243865967, 'eval_runtime': 4.4926, 'eval_samples_per_second': 112.629, 'eval_steps_per_second': 14.246, 'epoch': 0.7936507936507936}
{'loss': 2.3106, 'grad_norm': 33.6737174987793, 'learning_rate': 1.5222772277227723e-05, 'epoch': 1.1904761904761905}
{'loss': 1.8544, 'grad_norm': 45.748687744140625, 'learning_rate': 7.797029702970298e-06, 'epoch': 1.5873015873015874}
{'eval_loss': 2.1273882389068604, 'eval_runtime': 2.9173, 'eval_samples_per_second': 173.445, 'eval_steps_per_second': 21.938, 'epoch': 1.5873015873015874}
{'loss': 1.7333, 'grad_norm': 30.49672508239746, 'learning_rate': 3.712871287128713e-07, 'epoch': 1.9841269841269842}
{'train_runtime': 125.4824, 'train_samples_per_second': 32.132, 'train_steps_per_second': 4.016, 'train_loss

TrainOutput(global_step=504, training_loss=2.7333214850652787, metrics={'train_runtime': 125.4824, 'train_samples_per_second': 32.132, 'train_steps_per_second': 4.016, 'train_loss': 2.7333214850652787, 'epoch': 2.0})

In [53]:
metric = evaluate.load("squad")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # predictions are (start_logits, end_logits)
    start_preds = predictions[0].argmax(-1)
    end_preds = predictions[1].argmax(-1)

    # Convert to text answers
    pred_texts = []
    label_texts = []

    for i in range(len(start_preds)):
        input_ids = val_bert[i]["input_ids"]
        start, end = int(start_preds[i]), int(end_preds[i])
        pred_text = bert_tokenizer.decode(input_ids[start:end+1])
        pred_texts.append(pred_text)

        label_texts.append(val_small_dataset[i]["answers"]["text"][0])

    return metric.compute(predictions=pred_texts, references=label_texts)

# Example usage after training:
# results = trainer_bert.evaluate()
# print("BERT QA Results:", results)


In [54]:
question = "Who wrote the novel '1984'?"
context = "The novel '1984' was written by George Orwell in 1949."

inputs = bert_tokenizer(question, context, return_tensors="pt").to(device)
outputs = bert_model(**inputs)

start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1

answer = bert_tokenizer.convert_tokens_to_string(
    bert_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_idx:end_idx])
)

print("Predicted Answer:", answer)


Predicted Answer: george orwell


In [55]:
!pip install evaluate -q

import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)
import evaluate
from transformers import logging

logging.set_verbosity_error()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [56]:
# Load SQuAD v1.1 dataset
dataset = load_dataset("squad")

# Smaller subset for quick runs
train_size, val_size = 2000, 500
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(train_size))
small_val_dataset = dataset["validation"].shuffle(seed=42).select(range(val_size))

print("Example training record:", small_train_dataset[0])


Example training record: {'id': '573173d8497a881900248f0c', 'title': 'Egypt', 'context': 'The Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to a 2010 Pew Global Attitudes survey, 84% of Egyptians polled supported the death penalty for those who leave Islam; 77% supported whippings and cutting off of hands for theft and robbery; and 82% support stoning a person who commits adultery.', 'question': 'What percentage of Egyptians polled support death penalty for those leaving Islam?', 'answers': {'text': ['84%'], 'answer_start': [468]}}


In [57]:
t5_model_name = "t5-small"
t5_tokenizer = AutoTokenizer.from_pretrained(t5_model_name)

def preprocess_t5(examples):
    # Input: "question: ... context: ..."
    inputs = ["question: " + q + " context: " + c for q, c in zip(examples["question"], examples["context"])]
    targets = [a["text"][0] for a in examples["answers"]]

    model_inputs = t5_tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )

    labels = t5_tokenizer(
        targets, max_length=64, truncation=True, padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_t5 = small_train_dataset.map(preprocess_t5, batched=True, remove_columns=dataset["train"].column_names)
val_t5 = small_val_dataset.map(preprocess_t5, batched=True, remove_columns=dataset["validation"].column_names)

print("Sample tokenized T5 input:")
print(t5_tokenizer.decode(train_t5[0]["input_ids"][:100]))


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Sample tokenized T5 input:
question: What percentage of Egyptians polled support death penalty for those leaving Islam? context: The Pew Forum on Religion & Public Life ranks Egypt as the fifth worst country in the world for religious freedom. The United States Commission on International Religious Freedom, a bipartisan independent agency of the US government, has placed Egypt on its watch list of countries that require close monitoring due to the nature and extent of violations of religious freedom engaged in or tolerated by the government. According to 


In [58]:
# Data collator
data_collator_t5 = DataCollatorForSeq2Seq(tokenizer=t5_tokenizer, model=t5_model)

# Load T5 model
t5_model = AutoModelForSeq2SeqLM.from_pretrained(t5_model_name)

# Training arguments (compatible with your version)
training_args_t5 = TrainingArguments(
    output_dir="./t5-squad",
    eval_strategy="steps",            # <- fixed here
    eval_steps=100,
    logging_steps=100,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=3e-5,               # good default for T5
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=1,
    warmup_steps=50,
    gradient_accumulation_steps=4,
    fp16=torch.cuda.is_available(),
    report_to=[],  # disable wandb/tensorboard
)

trainer_t5 = Trainer(
    model=t5_model,
    args=training_args_t5,
    train_dataset=train_t5,
    eval_dataset=val_t5,
    tokenizer=t5_tokenizer,
    data_collator=data_collator_t5,
)

# Train (comment if just testing pipeline)
# trainer_t5.train()

/tmp/ipython-input-3740368195.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_t5 = Trainer(


In [59]:
question = "Who wrote the novel '1984'?"
context = "The novel '1984' was written by George Orwell in 1949."

input_text = "question: " + question + " context: " + context
inputs = t5_tokenizer(input_text, return_tensors="pt").to(device)

outputs = t5_model.generate(
    inputs["input_ids"],
    max_length=50,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95
)

print("Predicted Answer:", t5_tokenizer.decode(outputs[0], skip_special_tokens=True))


Predicted Answer: George Orwell


In [60]:
import evaluate

# Load official SQuAD metric (gives F1 and EM)
squad_metric = evaluate.load("squad")


In [61]:
def evaluate_gpt2(num_samples=20):
    preds, refs = [], []
    for i in range(num_samples):
        example = small_val_dataset[i]
        q = example["question"]
        c = example["context"]
        true_ans = example["answers"]

        input_text = f"question: {q} context: {c}"
        inputs = gpt2_tokenizer(input_text, return_tensors="pt").to(device)

        outputs = gpt2_model.generate(
            inputs["input_ids"],
            max_new_tokens=64,
            do_sample=False
        )
        pred_text = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)

        preds.append({"id": str(i), "prediction_text": pred_text})
        refs.append({"id": str(i), "answers": true_ans})

    return squad_metric.compute(predictions=preds, references=refs)

print("GPT-2 Evaluation:", evaluate_gpt2())


GPT-2 Evaluation: {'exact_match': 0.0, 'f1': 3.913101540426598}


In [62]:
def evaluate_bert(num_samples=20):
    preds, refs = [], []
    for i in range(num_samples):
        example = small_val_dataset[i]
        q = example["question"]
        c = example["context"]
        true_ans = example["answers"]

        inputs = bert_tokenizer(q, c, return_tensors="pt", truncation=True, max_length=384).to(device)
        outputs = bert_model(**inputs)

        start_idx = torch.argmax(outputs.start_logits)
        end_idx = torch.argmax(outputs.end_logits) + 1
        pred_text = bert_tokenizer.decode(inputs["input_ids"][0][start_idx:end_idx])

        preds.append({"id": str(i), "prediction_text": pred_text})
        refs.append({"id": str(i), "answers": true_ans})

    return squad_metric.compute(predictions=preds, references=refs)

print("BERT Evaluation:", evaluate_bert())


BERT Evaluation: {'exact_match': 40.0, 'f1': 48.444327731092436}


In [63]:
def evaluate_t5(num_samples=20):
    preds, refs = [], []
    for i in range(num_samples):
        example = small_val_dataset[i]
        q = example["question"]
        c = example["context"]
        true_ans = example["answers"]

        input_text = f"question: {q} context: {c}"
        inputs = t5_tokenizer(input_text, return_tensors="pt").to(device)

        outputs = t5_model.generate(
            inputs["input_ids"],
            max_new_tokens=64,
            do_sample=False
        )
        pred_text = t5_tokenizer.decode(outputs[0], skip_special_tokens=True)

        preds.append({"id": str(i), "prediction_text": pred_text})
        refs.append({"id": str(i), "answers": true_ans})

    return squad_metric.compute(predictions=preds, references=refs)

print("T5 Evaluation:", evaluate_t5())


T5 Evaluation: {'exact_match': 85.0, 'f1': 90.11904761904762}


In [72]:
import nbformat

# Replace with the name of your uploaded notebook file
input_file = "Gen_AI_Assignment_2.ipynb"
output_file = "Gen_AI_Assignment_2_cleaned.ipynb"

# Load notebook
with open(input_file, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Remove "widgets" metadata if present
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

# Save cleaned notebook
with open(output_file, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)

print(f"✅ Cleaned notebook saved as {output_file}")


FileNotFoundError: [Errno 2] No such file or directory: 'Gen_AI_Assignment_2.ipynb'